In [80]:
from selenium import webdriver
from bs4 import BeautifulSoup
import time

# WebDriver 경로 설정
# path = "path_to_chromedriver"  # 여기에 실제 ChromeDriver 경로를 입력해주세요.

# WebDriver 인스턴스 생성 (Chrome 브라우저)
driver = webdriver.Chrome()

# 페이지 로드
url = 'https://cafe.naver.com/joonggonara?iframe_url=/ArticleList.nhn%3Fsearch.clubid=10050146%26search.menuid=334%26search.boardtype=L%26search.totalCount=151%26search.cafeId=10050146%26search.page=1'
driver.get(url)

# 페이지가 완전히 로드될 때까지 대기
time.sleep(5)

# iframe으로 이동
driver.switch_to.frame('cafe_main')

# 페이지 소스를 가져옴
html = driver.page_source

# BeautifulSoup 객체 생성
soup = BeautifulSoup(html, 'html.parser')

# inner_number 찾기
inner_numbers = soup.find_all('div', {'class': 'inner_number'})

# WebDriver 종료
driver.quit()

for inner_number in inner_numbers:
    print(inner_number.text)


990920572
990920541
990920538
990920507
990920409
990920320
990920139
990920109
990920026
990919913
990919881
990919879
990919809
990919770
990919620


In [81]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import json
from selenium import webdriver
import time

def get_inner_numbers(page):
    # WebDriver 경로 설정
    # path = "path_to_chromedriver"  # 여기에 실제 ChromeDriver 경로를 입력

    # WebDriver 인스턴스 생성 (Chrome 브라우저)
    driver = webdriver.Chrome()

    # 페이지 로드
    url = f"https://cafe.naver.com/joonggonara?iframe_url=/ArticleList.nhn%3Fsearch.clubid=10050146%26search.menuid=334%26search.boardtype=L%26search.totalCount=151%26search.cafeId=10050146%26search.page={page}"
    driver.get(url)

    # 페이지가 완전히 로드될 때까지 대기
    time.sleep(5)

    # iframe으로 이동
    driver.switch_to.frame('cafe_main')

    # 페이지 소스를 가져옴
    html = driver.page_source

    # BeautifulSoup 객체 생성
    soup = BeautifulSoup(html, 'html.parser')

    # inner_number 찾기
    inner_numbers = soup.find_all('div', {'class': 'inner_number'})

    # WebDriver 종료
    driver.quit()

    # inner_numbers를 텍스트로 변환하여 리스트로 반환
    return [num.text.strip() for num in inner_numbers]

def ju(page):
    # API URL을 쉽게 바꿀 수 있도록 만든 base url
    base_api_url = "https://apis.naver.com/cafe-web/cafe-articleapi/v2.1/cafes/10050146/articles/{}?query=&menuId=334&boardType=L&useCafeId=true&requestFrom=A"

    # inner_number 가져오기
    inner_numbers = get_inner_numbers(page)

    # API를 사용하여 데이터 가져오기
    data_list = []
    for num in inner_numbers:
        api_url = base_api_url.format(num)
        api_res = requests.get(api_url)
        api_data = api_res.json()  # API response to json

        # 'result'가 존재하고 그 아래에 'article'이 존재하는지 확인
        if 'result' in api_data and 'article' in api_data['result']:
            # 원하는 데이터를 추출하고 이를 data_list에 추가
            data = api_data['result']['article']  # JSON 구조에 따라 수정
        else:
            # 'article' 정보가 없는 경우에 대비해 빈 딕셔너리를 사용
            data = {}
        # inner_number를 추가하고, 없는 값에 대해서는 공백으로 처리
        data_list.append({
            'inner_number': num,
            'articleId': data.get('id', ''),
            'subject': data.get('subject', ''),
            'writer_id': data.get('writer', {}).get('id', ''),
            'writer_nick': data.get('writer', {}).get('nick', ''),
            'memberLevelName': data.get('writer', {}).get('memberLevelName', ''),
            'writeDate': data.get('writeDate', ''),
            'readCount': data.get('readCount', ''),
            'commentCount': data.get('commentCount', ''),
        })

    # 데이터프레임으로 변환 후 csv 파일로 저장
    df = pd.DataFrame(data_list)
    df.to_csv("data.csv", index=False)

In [82]:
ju(5)

In [85]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import json
from selenium import webdriver
import time

def get_inner_numbers(page):
    # WebDriver 경로 설정
    # path = "path_to_chromedriver"  # 여기에 실제 ChromeDriver 경로를 입력

    # WebDriver 인스턴스 생성 (Chrome 브라우저)
    driver = webdriver.Chrome()

    # 페이지 로드
    url = f"https://cafe.naver.com/joonggonara?iframe_url=/ArticleList.nhn%3Fsearch.clubid=10050146%26search.menuid=334%26search.boardtype=L%26search.totalCount=151%26search.cafeId=10050146%26search.page={page}"
    driver.get(url)

    # 페이지가 완전히 로드될 때까지 대기
    time.sleep(5)

    # iframe으로 이동
    driver.switch_to.frame('cafe_main')

    # 페이지 소스를 가져옴
    html = driver.page_source

    # BeautifulSoup 객체 생성
    soup = BeautifulSoup(html, 'html.parser')

    # inner_number 찾기
    inner_numbers = soup.find_all('div', {'class': 'inner_number'})

    # WebDriver 종료
    driver.quit()

    # inner_numbers를 텍스트로 변환하여 리스트로 반환
    return [num.text.strip() for num in inner_numbers]

def ju(inner_numbers):
    # API URL을 쉽게 바꿀 수 있도록 만든 base url
    base_api_url = "https://apis.naver.com/cafe-web/cafe-articleapi/v2.1/cafes/10050146/articles/{}?query=&menuId=334&boardType=L&useCafeId=true&requestFrom=A"

    # API를 사용하여 데이터 가져오기
    data_list = []
    for num in inner_numbers:
        api_url = base_api_url.format(num)
        api_res = requests.get(api_url)
        api_data = api_res.json()  # API response to json

        # 'result'가 존재하고 그 아래에 'article'이 존재하는지 확인
        if 'result' in api_data and 'article' in api_data['result']:
            # 원하는 데이터를 추출하고 이를 data_list에 추가
            data = api_data['result']['article']  # JSON 구조에 따라 수정
        else:
            # 'article' 정보가 없는 경우에 대비해 빈 딕셔너리를 사용
            data = {}
        # inner_number를 추가하고, 없는 값에 대해서는 공백으로 처리
        data_list.append({
            'inner_number': num,
            'articleId': data.get('id', ''),
            'subject': data.get('subject', ''),
            'writer_id': data.get('writer', {}).get('id', ''),
            'writer_nick': data.get('writer', {}).get('nick', ''),
            'memberLevelName': data.get('writer', {}).get('memberLevelName', ''),
            'writeDate': data.get('writeDate', ''),
            'readCount': data.get('readCount', ''),
            'commentCount': data.get('commentCount', ''),
        })

    # 데이터프레임으로 변환 후 csv 파일로 저장
    df = pd.DataFrame(data_list)
    df.to_csv("data.csv", index=False)

# 여러 페이지의 데이터 가져오기
start_page = 1
end_page = 5  # 마지막 페이지 번호 설정

all_inner_numbers = []
for page in range(start_page, end_page+1):
    inner_numbers = get_inner_numbers(page)
    all_inner_numbers.extend(inner_numbers)

# API 정보 가져오기
ju(all_inner_numbers)
